# TO BE DONE

In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
df_yake_uspto_epo_rel = pd.read_json('/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/df_keywords_list_agg_uspto_epo_rel_embeddings_noun_chunks.json')
# Drop columns keyword_yake_pos, keyword_yake_patentsberta_embedding, keyword_yake_climatebert_embedding and keyword_yake_bertforpatents_embedding
df_yake_uspto_epo_rel.drop(columns=['keyword_yake_patentsberta_embedding', 'keyword_yake_climatebert_embedding', 'keyword_yake_bertforpatents_embedding'], inplace=True)

In [3]:
df_yake_cleantech_titles = pd.read_json('/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/cpc_yake_keywords_list_noun_chunks_embeddings.json')

In [4]:
# Extract unique keywords
cleantech_titles = df_yake_cleantech_titles['keyword_yake_lemma'].unique()
uspto_epo_rel_keywords = df_yake_uspto_epo_rel['keyword_yake_lemma'].unique()

# Initialize co-occurrence matrix
co_occurrence_matrix = pd.DataFrame(0, index=cleantech_titles, columns=uspto_epo_rel_keywords)

In [5]:
# Exploding the 'patent_id' column while retaining the 'keyword_yake' column
patent_id_exploded = df_yake_uspto_epo_rel.set_index('keyword_yake_lemma')['patent_id'].explode().reset_index()
# Put 'US' in front of each patent_id
patent_id_exploded['patent_id'] = 'US' + patent_id_exploded['patent_id'].astype(str)
# Delete all rows where patent_id contains the string None
patent_id_exploded = patent_id_exploded[~patent_id_exploded['patent_id'].str.contains('None')]
# Aggregate on 'patent_id', list of keyword_yake_lemma
patent_id_exploded = patent_id_exploded.groupby('patent_id')['keyword_yake_lemma'].apply(list).reset_index()

In [6]:
patent_id_exploded.head()

,patent_id,keyword_yake_lemma
0,US10000011,[desired]
1,US10000017,"[attachment, inlet opening, mounting apparatus..."
2,US10000021,"[machining process, predetermined portion, sha..."
3,US10000025,"[damage tolerance, fiber orientation]"
4,US10000033,"[closure element, eva, substantially flat]"


In [7]:
# Exploding the 'publn_nr' column while retaining the 'keyword_yake' column
publn_nr_exploded = df_yake_uspto_epo_rel.set_index('keyword_yake_lemma')['publn_nr'].explode().reset_index()
# Put 'EP' in front of each publn_nr
publn_nr_exploded['publn_nr'] = 'EP' + publn_nr_exploded['publn_nr'].astype(str)
# Delete all rows where publn_nr contains the string None
publn_nr_exploded = publn_nr_exploded[~publn_nr_exploded['publn_nr'].str.contains('None')]
# Aggregate on 'publn_nr', list all keyword_yake_lemma
publn_nr_exploded = publn_nr_exploded.groupby('publn_nr')['keyword_yake_lemma'].apply(list).reset_index()

In [8]:
publn_nr_exploded.head()

,publn_nr,keyword_yake_lemma
0,EP0000085,"[benzene, dilute solution, hbr, tributyl]"
1,EP0000338,"[agar gel, bluetongue virus, charlottesville, ..."
2,EP0000401,"[flock, livelihood, main source, negev, rumina..."
3,EP0000715,"[high efficiency solar, large area photovoltai..."
4,EP0000785,[drainage]


In [9]:
# Exploding the 'oaid' column while retaining the 'keyword_yake' column
oaid_exploded = df_yake_uspto_epo_rel.set_index('keyword_yake_lemma')['oaid'].explode().reset_index()
# Put 'REL' in front of each oaid
oaid_exploded['oaid'] = 'REL' + oaid_exploded['oaid'].astype(str)
# Delete all rows where oaid contains the string None
oaid_exploded = oaid_exploded[~oaid_exploded['oaid'].str.contains('None')]
# Aggregate on 'oaid', list of 'keyword_yake_lemma'
oaid_exploded = oaid_exploded.groupby('oaid')['keyword_yake_lemma'].apply(list).reset_index()

In [10]:
oaid_exploded.head()

,oaid,keyword_yake_lemma
0,REL100000185,"[pain, physiology]"
1,REL1000054809,"[distributed, droop control, electric power sy..."
2,REL100007697,"[latency, millisecond, problematic, rcu, sched..."
3,REL100014517,"[biochemical and genetic, circular form, idea,..."
4,REL1000192467,"[arthralgia, headache, july, oval, pregnant wo..."


In [42]:
# Concatenate the three exploded dataframes
df_keywords_uspto_epo_rel = pd.concat([patent_id_exploded, publn_nr_exploded, oaid_exploded], axis=0).reset_index(drop=True)

In [43]:
df_keywords_uspto_epo_rel.sample(5)

,patent_id,keyword_yake_lemma,publn_nr,oaid
228530,US5915816,"[adsorbent, blower, cooling zone, heat accumul...",NaN,NaN
621479,NaN,"[positive number, single crystal phosphor, vis...",EP3140368,NaN
900664,NaN,"[celluclast, total sugar]",NaN,REL2031919653
342626,US8087248,"[dust, heated working fluid, hot gas, temperat...",NaN,NaN
1169435,NaN,"[birth, infant, milk]",NaN,REL2162304021


In [13]:
# Explode the 'keyword_yake_lemma' column
df_keywords_uspto_epo_rel_exploded = df_keywords_uspto_epo_rel.explode('keyword_yake_lemma').reset_index(drop=True)

In [28]:
# Merge df_keywords_uspto_epo_rel_exploded with df_yake_cleantech_titles on 'keyword_yake_lemma'and keep patent_id, publn_nr and oaid, including duplicates
df_yake_uspto_epo_rel_titles_filtered = pd.merge(df_yake_cleantech_titles, df_keywords_uspto_epo_rel_exploded, how='inner', left_on='keyword_yake_lemma', right_on='keyword_yake_lemma', validate='many_to_many')

In [29]:
df_yake_uspto_epo_rel_titles_filtered['keyword_yake_lemma'].nunique()

205

## Test different Keywords

In [56]:
# Print all rows where keyword = xxx in df_yake_uspto_epo_rel_titles_filtered
df_yake_uspto_epo_rel_titles_test = df_yake_uspto_epo_rel_titles_filtered[df_yake_uspto_epo_rel_titles_filtered['keyword_yake_lemma'] == 'wind energy']
df_yake_uspto_epo_rel_titles_test.reset_index(drop=True, inplace=True)

In [1]:
len(df_yake_uspto_epo_rel_titles_test)

NameError: name 'df_yake_uspto_epo_rel_titles_test' is not defined

In [58]:
for index, row in tqdm(df_yake_uspto_epo_rel_titles_filtered.iterrows()):
    # Check if patent_id, publn_nr or oaid are type string
    if isinstance(row['patent_id'], str):
        # Handle list of keywords in 'keyword_yake'
        for keyword_list in df_keywords_uspto_epo_rel[df_keywords_uspto_epo_rel['patent_id'] == row['patent_id']]['keyword_yake_lemma']:
            for keyword in keyword_list:
                co_occurrence_matrix.loc[row['keyword_yake_lemma'], keyword] += 1
    elif isinstance(row['publn_nr'], str):
        # Handle list of keywords in 'keyword_yake'
        for keyword_list in df_keywords_uspto_epo_rel[df_keywords_uspto_epo_rel['publn_nr'] == row['publn_nr']]['keyword_yake_lemma']:
            for keyword in keyword_list:
                co_occurrence_matrix.loc[row['keyword_yake_lemma'], keyword] += 1
    elif isinstance(row['oaid'], str):
        # Handle list of keywords in 'keyword_yake'
        for keyword_list in df_keywords_uspto_epo_rel[df_keywords_uspto_epo_rel['oaid'] == row['oaid']]['keyword_yake_lemma']:
            for keyword in keyword_list:
                co_occurrence_matrix.loc[row['keyword_yake_lemma'], keyword] += 1
    else:
        print('Error')

179it [00:23,  7.75it/s]


In [ ]:
# Divide rows in co-occurence matrix by largest value in row
co_occurrence_matrix = co_occurrence_matrix.div(co_occurrence_matrix.max(axis=1), axis=0)

In [ ]:
co_occurence_matrix.to_csv('/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/co_occurrence_matrix.csv')